In [1]:
# 中文财经

In [9]:
import requests
from bs4 import BeautifulSoup
import re

In [10]:
# 根据给定url链接 返回对应页面的HTML
def getHTML(url):
    s = requests.session()
    s.keep_alive = False
    r = requests.get(url, timeout=30, headers={'Connection':'close'}, stream=True, verify=False)
    r.encoding = r.apparent_encoding
    return r.text

In [11]:
# 东亚日报 财经频道地址
srcUrl = 'https://www.donga.com/cn/List?c=02'

In [12]:
# 获取文章列表中每篇文章的链接地址
srcHtml = getHTML(srcUrl)

E:\Anaconda3\lib\site-packages\urllib3\connectionpool.py:981: InsecureRequestWarning: Unverified HTTPS request is being made to host 'www.donga.com'. Adding certificate verification is strongly advised. See: https://urllib3.readthedocs.io/en/latest/advanced-usage.html#ssl-warnings
  warnings.warn(


In [13]:
# 分析HTML页面获取对应链接地址列表
soup = BeautifulSoup(srcHtml, 'html.parser')
all_a_href = soup.find_all('a')

linkLists = []
for href in all_a_href:
    temp = str(href)
    matchObj = re.match(r'<a href="https://www.donga.com/cn/List/article/all/', temp, re.M|re.I)
    if matchObj:
        linkLists.append(temp)

In [14]:
# 处理得到url
for i in range(len(linkLists)):
    temp = linkLists[i]
    temp = re.sub(r'<a href="', "", temp)
    temp = re.sub(r'">.*', "", temp)
    #print(temp)
    linkLists[i] = temp

# 去重
linkLists = list(set(linkLists))

In [15]:
for url in linkLists:
    print(url)

https://www.donga.com/cn/List/article/all/20210319/2511760/1/13年不变的房产综合税-9亿韩元-标准，是时候重新讨论了
https://www.donga.com/cn/List/article/all/20210310/2490166/1/美国国债利率暴涨冲击，国内股市动荡
https://www.donga.com/cn/List/article/all/20210320/2514526/1/现代重工业举行郑周永会长20周年追悼会
https://www.donga.com/cn/List/article/all/20210316/2503933/1/起亚首次公开配备E-GMP的电动汽车-EV6-设计
https://www.donga.com/cn/List/article/all/20210308/2484916/1/特斯拉股价跌破600美元大关-西学蚂蚁的-不眠之夜
https://www.donga.com/cn/List/article/all/20210310/2490226/1/海外股票投资创历史最高纪录，现在是防范利率、油价暴涨的时候了
https://www.donga.com/cn/List/article/all/20210309/2487379/1/1月饭桌物价上涨6-5-，上涨率居OECD第4位
https://www.donga.com/cn/List/article/all/20210313/2498290/1/LG为新建美国电池工厂投资5万亿韩元以上
https://www.donga.com/cn/List/article/all/20210311/2493145/1/现代汽车首次公开-高性能SUV-科纳N预告图
https://www.donga.com/cn/List/article/all/20210313/2498302/1/外卖的民族-金峰镇议长向餐饮业主支援200亿韩元的个人财产
https://www.donga.com/cn/List/article/all/20210318/2509240/1/洪楠基副总理称-就业市场困难得到缓解-难道他听不到二三十岁年轻人的呐喊声吗？
https://www.donga.com/cn/List/article/all

In [16]:
# 获取对应文章内容
def getContent(url):
    html = getHTML(url)
    # 分析html文件
    # 文章内容在 <div class="news_view">内
    # 通过正则处理获取文章内容
    res = re.findall(r' <div class="news_view"(.*?)</div>', html)
    res = str(res)
    res = re.sub(r'id(.*?)<br>', "", res)
    res = re.sub(r'<br>', '\n', res)
    return res

In [17]:
# 将对应文章保存至文件
# 文件名为 日期和文件编号
def getText(url):
    content = getContent(url)
    arr = url.split("/")
    fileName = arr[-4] + "-" + arr[-3]
    file = open("text/" + fileName + ".txt", "w+", encoding='utf-8')
    file.write(content)
    file.close()

In [37]:
# 测试
#getText(linkLists[0])

In [57]:
for i in range(len(linkLists)):
    getText(linkLists[i])
    
    if i == 4:
        break

In [39]:
#print(srcHtml)

In [ ]:
# Request URL: https://www.donga.com/cn/List?p0=j&p=21&l=20&c=02
# Request URL: https://www.donga.com/cn/List?p0=j&p=41&l=20&c=02

In [40]:
newPage = getHTML('https://www.donga.com/cn/List?p0=j&p=21&l=20&c=02')
#print(newPage)

In [15]:
# 通过浏览器分析 更多按钮 
# 发现每次点击后会发起新 请求request URL：https://www.donga.com/cn/List?p0=j&p=21&l=20&c=02 
# 且请求的依次变化为 https://www.donga.com/cn/List?p0=j&p=41&l=20&c=02 
# https://www.donga.com/cn/List?p0=j&p=61&l=20&c=02 

In [18]:
# 通过first+second+third 拼接每次more request的url
# first = "https://www.donga.com/cn/List?p0=j&p="
# second = 2
# third = "1&l=20&c=02"

# 下面通过函数实现获取下一次的url
def getNextClickMoreUrl():
    if not hasattr(getNextClickMoreUrl, 'second'):
        getNextClickMoreUrl.second = 0
    first = "https://www.donga.com/cn/List?p0=j&p="
    third = "1&l=20&c=02"
    getNextClickMoreUrl.second += 2
    return first + str(getNextClickMoreUrl.second) + third

In [19]:
#print(getNextClickMoreUrl())
#print(getNextClickMoreUrl())
#print(getNextClickMoreUrl())
#print(getNextClickMoreUrl())
#print(getNextClickMoreUrl())

In [20]:
# 通过生成的click more 的url获取新得到的相关文章的url
def getUrlAfterClickMore(url):
    html = getHTML(url)
    sp = BeautifulSoup(html, 'html.parser')
    all_a_href = sp.find_all('a')
    
    linkLists = []

    for href in all_a_href:
        temp = str(href)
        matchObj = re.match(r'<a href="https://www.donga.com/cn/List/article/all/', temp, re.M|re.I)
        if matchObj:# 符合条件 是所需的文章的URL
            temp = re.sub(r'<a href="', "", temp)
            temp = re.sub(r'">.*', "", temp)
            linkLists.append(temp)
    # 去重        
    linkLists = list(set(linkLists))
    return linkLists

In [38]:
newPageUrls = getUrlAfterClickMore('https://www.donga.com/cn/List?p0=j&p=21&l=20&c=02')

for url in newPageUrls:
    print(url)

https://www.donga.com/cn/List/article/all/20210219/2446990/1/股票代替零花钱-，未成年账户1年内翻番
https://www.donga.com/cn/List/article/all/20210223/2455363/1/特斯拉-5999万韩元-Model-Y突然停售
https://www.donga.com/cn/List/article/all/20210227/2465719/1/韩华集团会长金升渊3月时隔7年重返经营舞台
https://www.donga.com/cn/List/article/all/20210222/2452888/1/现代汽车集团名誉会长郑梦九完全退出经营层
https://www.donga.com/cn/List/article/all/20210306/2481667/1/美国ITC认定：-SK侵犯LG-22项商业秘密
https://www.donga.com/cn/List/article/all/20210305/2478793/1/高档的内饰-就像来到了头等舱休息室一样
https://www.donga.com/cn/List/article/all/20210301/2469073/1/汽车半导体大乱-，现代汽车蔚山工厂也-减少加班
https://www.donga.com/cn/List/article/all/20210226/2463064/1/公共设施的电动汽车充电机所有人都可以使用
https://www.donga.com/cn/List/article/all/20210220/2449552/1/SK会长崔泰源：-在电池生态系统合作非常重要
https://www.donga.com/cn/List/article/all/20210302/2470795/1/三星-Neo-QLED-TV-在美国被称赞为-最佳产品
https://www.donga.com/cn/List/article/all/20210216/2439769/1/LG与SK的电池争端，国家未来竞争力取决于后续协议
https://www.donga.com/cn/List/article/all/20210226/2463049/1/脸书：-3年内将在利用新闻资讯

In [21]:
import time

In [22]:
# 将list数据写入文件
def listToText(listValue):
    # 文件名用此时时间戳
    fileName = str(time.time())
    
    with open("pageUrl/" + fileName + ".txt", 'w+', encoding='utf-8') as f:
        f.write('\n'.join(listValue))

In [61]:
for i in range(5):
    url = getNextClickMoreUrl()
    linkLists = getUrlAfterClickMore(url)
    listToText(linkLists)# 将获取到的每篇文章的url写入文件

In [63]:
# 获取对应文章内容
# print(getContent('https://www.donga.com/cn/List/article/all/20210222/2452933/1/首尔市将老旧柴油车报废补贴提高到600万韩元'))

In [23]:
# 爬取两个页面所有文章数据
for i in range(2):
    linkLists = getUrlAfterClickMore(getNextClickMoreUrl())
    print("getLinkList    " + str(i))
    for link in linkLists:
        print("get Page")
        getText(link)

E:\Anaconda3\lib\site-packages\urllib3\connectionpool.py:981: InsecureRequestWarning: Unverified HTTPS request is being made to host 'www.donga.com'. Adding certificate verification is strongly advised. See: https://urllib3.readthedocs.io/en/latest/advanced-usage.html#ssl-warnings
  warnings.warn(


getLinkList    0
get Page


E:\Anaconda3\lib\site-packages\urllib3\connectionpool.py:981: InsecureRequestWarning: Unverified HTTPS request is being made to host 'www.donga.com'. Adding certificate verification is strongly advised. See: https://urllib3.readthedocs.io/en/latest/advanced-usage.html#ssl-warnings
  warnings.warn(


get Page


E:\Anaconda3\lib\site-packages\urllib3\connectionpool.py:981: InsecureRequestWarning: Unverified HTTPS request is being made to host 'www.donga.com'. Adding certificate verification is strongly advised. See: https://urllib3.readthedocs.io/en/latest/advanced-usage.html#ssl-warnings
  warnings.warn(


...

E:\Anaconda3\lib\site-packages\urllib3\connectionpool.py:981: InsecureRequestWarning: Unverified HTTPS request is being made to host 'www.donga.com'. Adding certificate verification is strongly advised. See: https://urllib3.readthedocs.io/en/latest/advanced-usage.html#ssl-warnings
  warnings.warn(


get Page


E:\Anaconda3\lib\site-packages\urllib3\connectionpool.py:981: InsecureRequestWarning: Unverified HTTPS request is being made to host 'www.donga.com'. Adding certificate verification is strongly advised. See: https://urllib3.readthedocs.io/en/latest/advanced-usage.html#ssl-warnings
  warnings.warn(


get Page


E:\Anaconda3\lib\site-packages\urllib3\connectionpool.py:981: InsecureRequestWarning: Unverified HTTPS request is being made to host 'www.donga.com'. Adding certificate verification is strongly advised. See: https://urllib3.readthedocs.io/en/latest/advanced-usage.html#ssl-warnings
  warnings.warn(
